In [ ]:
!pip3 install codeshield
import json
import os
from tqdm import tqdm
from codeshield.cs import CodeShield, Language
import pandas as pd

LANGUAGE_MAP = {
    "python": Language.PYTHON,
    "c": Language.C,
    "cpp": Language.CPP,
    "java": Language.JAVA,
}

!pip install semgrep --quiet

!semgrep login

# Evaluation for Gemini 1.5 Pro

In [ ]:
import os
import json
import subprocess
import pandas as pd
from tqdm import tqdm
import nest_asyncio
import asyncio
from codeshield.cs import CodeShield, Language

nest_asyncio.apply()

LANGUAGE_MAP = {
    "c": Language.C,
}

CONFIGS = {
    "ext": {"python": "py", "java": "java", "c": "c", "cpp": "cpp"},
    "config": {
        "c": "r/c",
        "cpp": "r/c",
        "java": "p/security-audit",
        "python": "p/security-audit"
    }
}

LANGUAGE_FILES = {
    "c": "/kaggle/input/evaluatesercurecode/Final_Gemini_Pro/phase2_end_code_Gemini_c_sce2.csv",
}

def install_semgrep():
    subprocess.run(["pip", "install", "semgrep", "--quiet"], check=True)

def write_source_files(df, out_dir, ext):
    import shutil
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir, exist_ok=True)
    file_list = []
    for idx, row in df.iterrows():
        filename = os.path.join(out_dir, f"file_{idx}.{ext}")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(row.get("fixed_code_gemini", ""))
        file_list.append((idx, filename))
    return file_list

def run_semgrep(out_dir, config, lang):
    result_file = os.path.join(out_dir, f"security_audit_results_{lang}.json")
    subprocess.run([
        "semgrep", "scan",
        f"--config={config}",
        "--json",
        f"--json-output={result_file}",
        out_dir
    ], check=True)
    return result_file

def analyze_semgrep_results(result_file, total_files):
    with open(result_file, "r", encoding='utf-8') as f:
        results = json.load(f)
    findings = results.get("results", [])
    insecure_files = set(int(os.path.basename(item["path"]).split("_")[1].split(".")[0]) for item in findings)
    return insecure_files

async def scan_llm_output(code_snippets, lang_enum):
    insecure_indices = []
    for idx, code in tqdm(enumerate(code_snippets), total=len(code_snippets), desc="CodeShield Scanning"):
        result = await CodeShield.scan_code(code)
        if result.is_insecure:
            insecure_indices.append(idx)
    return set(insecure_indices)

async def run_combined_security_pipeline():
    install_semgrep()

    for lang_key, lang_enum in LANGUAGE_MAP.items():
        print(f"\n🔍 Processing: {lang_key.upper()}")
        code_path = LANGUAGE_FILES[lang_key]
        if not os.path.exists(code_path):
            print(f"❌ File not found: {code_path}")
            continue

        df = pd.read_csv(code_path)
        code_list = df["fixed_code_gemini"].tolist()
        total_snippets = len(code_list)

        # --- CodeShield Scan ---
        print("🛡️ Running CodeShield scan...")
        cs_insecure_ids = await scan_llm_output(code_list, lang_enum)
        cs_df = pd.DataFrame({
            "code_id": list(cs_insecure_ids),
            "codeshield_insecure": True
        })

        # --- Semgrep Scan ---
        print("🔍 Running Semgrep scan...")
        ext = CONFIGS["ext"][lang_key]
        semgrep_config = CONFIGS["config"][lang_key]
        file_list = write_source_files(df, f"{lang_key}_temp", ext)
        semgrep_result_file = run_semgrep(f"{lang_key}_temp", semgrep_config, lang_key)
        semgrep_insecure_ids = analyze_semgrep_results(semgrep_result_file, total_snippets)
        semgrep_df = pd.DataFrame({
            "code_id": list(semgrep_insecure_ids),
            "semgrep_insecure": True
        })

        # --- Base DataFrame for Merging ---
        base_df = pd.DataFrame({"code_id": list(range(total_snippets))})

        # --- Merge Results ---
        merged_df = base_df.merge(cs_df, on="code_id", how="left")
        merged_df = merged_df.merge(semgrep_df, on="code_id", how="left")

        merged_df["codeshield_insecure"] = merged_df["codeshield_insecure"].fillna(False)
        merged_df["semgrep_insecure"] = merged_df["semgrep_insecure"].fillna(False)
        merged_df["is_secure"] = ~(merged_df["codeshield_insecure"] | merged_df["semgrep_insecure"])

        # --- Compute Statistics ---
        secure_count = merged_df["is_secure"].sum()
        insecure_count = total_snippets - secure_count
        secure_percent = (secure_count / total_snippets) * 100 if total_snippets else 0

        # --- Save and Report ---
        output_path = f"combined_security_analysis_{lang_key}.csv"
        merged_df.to_csv(output_path, index=False)

        print(f"\n📊 {lang_key.upper()} SCAN SUMMARY")
        print(f" - Total code snippets: {total_snippets}")
        print(f" - CodeShield insecure: {len(cs_insecure_ids)}")
        print(f" - Semgrep insecure: {len(semgrep_insecure_ids)}")
        print(f" - 🔐 Secure snippets: {secure_count}")
        print(f" - ✅ Secure percentage: {secure_percent:.2f}%")
        print(f" - 📁 Output saved to: {output_path}")

# --- Execute in Notebook/Kaggle environment ---
await run_combined_security_pipeline()


# Evaluation for CodeLLama

In [ ]:
import os
import json
import subprocess
import pandas as pd
from tqdm import tqdm
import nest_asyncio
import asyncio
from codeshield.cs import CodeShield, Language

nest_asyncio.apply()

LANGUAGE_MAP = {
    "python": Language.PYTHON,
}

CONFIGS = {
    "ext": {"python": "py", "java": "java", "c": "c", "cpp": "cpp"},
    "config": {"c": "r/c", "cpp": "r/c", "java": "p/security-audit", "python": "p/security-audit"}
}

LANGUAGE_FILES = {
    "python": "/kaggle/input/evaluatesercurecode/python_fixed_code_output.json",
}

def install_semgrep():
    subprocess.run(["pip", "install", "semgrep", "--quiet"], check=True)

def write_source_files(df, out_dir, ext):
    import shutil
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir, exist_ok=True)
    file_list = []
    for idx, row in df.iterrows():
        filename = os.path.join(out_dir, f"file_{idx}.{ext}")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(row.get("fixed_code_codellama", ""))
        file_list.append((idx, filename))
    return file_list

def run_semgrep(out_dir, config, lang):
    result_file = os.path.join(out_dir, f"security_audit_results_{lang}.json")
    subprocess.run([
        "semgrep", "scan",
        f"--config={config}",
        "--json",
        f"--json-output={result_file}",
        out_dir
    ], check=True)
    return result_file

def analyze_semgrep_results(result_file, total_files):
    with open(result_file, "r", encoding='utf-8') as f:
        results = json.load(f)
    findings = results.get("results", [])
    insecure_files = set(int(os.path.basename(item["path"]).split("_")[1].split(".")[0]) for item in findings)
    return insecure_files

async def scan_llm_output(code_snippets, lang_enum):
    insecure_indices = []
    for idx, code in tqdm(enumerate(code_snippets), total=len(code_snippets), desc="CodeShield Scanning"):
        result = await CodeShield.scan_code(code)
        if result.is_insecure:
            insecure_indices.append(idx)
    return set(insecure_indices)

async def run_combined_security_pipeline():
    install_semgrep()

    for lang_key, lang_enum in LANGUAGE_MAP.items():
        print(f"\n🔍 Processing: {lang_key.upper()}")
        code_path = LANGUAGE_FILES[lang_key]
        if not os.path.exists(code_path):
            print(f"❌ File not found: {code_path}")
            continue

        # --- Load JSON file instead of CSV ---
        with open(code_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        df = pd.json_normalize(data)
        df["fixed_code_codellama"] = df["fixed_code"]  # prepare correct column

        code_list = df["fixed_code_codellama"].tolist()
        total_snippets = len(code_list)

        # --- CodeShield Scan ---
        print("🛡️ Running CodeShield scan...")
        cs_insecure_ids = await scan_llm_output(code_list, lang_enum)
        cs_df = pd.DataFrame({
            "code_id": list(cs_insecure_ids),
            "codeshield_insecure": True
        })

        # --- Semgrep Scan ---
        print("🔍 Running Semgrep scan...")
        ext = CONFIGS["ext"][lang_key]
        semgrep_config = CONFIGS["config"][lang_key]
        file_list = write_source_files(df, f"{lang_key}_temp", ext)
        semgrep_result_file = run_semgrep(f"{lang_key}_temp", semgrep_config, lang_key)
        semgrep_insecure_ids = analyze_semgrep_results(semgrep_result_file, total_snippets)
        semgrep_df = pd.DataFrame({
            "code_id": list(semgrep_insecure_ids),
            "semgrep_insecure": True
        })

        # --- Merge Results ---
        base_df = pd.DataFrame({"code_id": list(range(total_snippets))})
        merged_df = base_df.merge(cs_df, on="code_id", how="left")
        merged_df = merged_df.merge(semgrep_df, on="code_id", how="left")
        merged_df["codeshield_insecure"] = merged_df["codeshield_insecure"].fillna(False)
        merged_df["semgrep_insecure"] = merged_df["semgrep_insecure"].fillna(False)
        merged_df["is_secure"] = ~(merged_df["codeshield_insecure"] | merged_df["semgrep_insecure"])

        # --- Output Summary ---
        secure_count = merged_df["is_secure"].sum()
        insecure_count = total_snippets - secure_count
        secure_percent = (secure_count / total_snippets) * 100 if total_snippets else 0
        output_path = f"combined_security_analysis_{lang_key}.csv"
        merged_df.to_csv(output_path, index=False)

        print(f"\n📊 {lang_key.upper()} SCAN SUMMARY")
        print(f" - Total code snippets: {total_snippets}")
        print(f" - CodeShield insecure: {len(cs_insecure_ids)}")
        print(f" - Semgrep insecure: {len(semgrep_insecure_ids)}")
        print(f" - 🔐 Secure snippets: {secure_count}")
        print(f" - ✅ Secure percentage: {secure_percent:.2f}%")
        print(f" - 📁 Output saved to: {output_path}")

# --- Run the pipeline ---
await run_combined_security_pipeline()
